In [1]:
# Import Libraries 
import pandas as pd 
import numpy as np
import os

In [2]:
# Create Path 
path = r"C:\Users\Megan\Documents\CareerFoundry\Unit 4\InstaCart Python Project"

In [3]:
# Import dataframe 
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'merge_loyalty.pkl'))

## 02. Find aggregated mean of 'order_number' column, grouped by 'department_id' 

In [4]:
# Find mean of order_number (grouped by department_id) for the whole dataframe
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [5]:
# Create subset for comparison 
df = ords_prods_merge[:1000000]


In [6]:
# Find mean of order numbers for subset 
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


## 03. Analyze the results and compare with mean of the subset dataframe

### In the dataframe subset (df), not all of the departments are represented in the results. When applying the aggregation to the entire dataframe, all the departments are represented. The mean for each department changed between the subset and the entire df as well, since there are much more records to average in the entire dataframe. 


## 04. Follow the instructions for creating a loyalty flag for existing customers using the transform() and loc() functions 

In [7]:
# Split data on user_id column, create new max_order column and transform on order_number to find max orders
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [8]:
# Check output 
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,Busiest Days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,11.0,196,1,0,both,Soda,77,7,9.0,Mid-Range Product,Regularly Busy,Average Orders,10,New Customer
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,New Customer
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,New Customer
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,New Customer
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,New Customer


In [9]:
# Drop previous loyalty column (to make a slightly different, new one)
ords_prods_old = ords_prods_merge.drop(columns = ['loyalty_flag'])

In [10]:
ords_prods_old.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,Busiest Days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,11.0,196,1,0,both,Soda,77,7,9.0,Mid-Range Product,Regularly Busy,Average Orders,10
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10


In [11]:
# Use loc function to create loyalty column for orders over 35 
ords_prods_old.loc[ords_prods_old['max_order'] > 35, 'loyalty_flag'] = 'Loyal Customer'

In [12]:
# Use loc function to create 'regular customer' loyalty flag 
ords_prods_old.loc[(ords_prods_old['max_order'] <= 35) & (ords_prods_old['max_order'] >= 10), 'loyalty_flag'] = 'Regular Customer'

In [13]:
# Use loc function to create 'new customer' loyalty flag 
ords_prods_old.loc[ords_prods_old['max_order'] < 10, 'loyalty_flag'] = 'New Customer'

In [14]:
# Check value counts 
ords_prods_old['loyalty_flag'].value_counts(dropna = False)

Regular Customer    14803614
Loyal Customer      12130379
New Customer         5470866
Name: loyalty_flag, dtype: int64

## 05. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer).

In [15]:
ords_prods_old.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal Customer    10.690627  1.0  99999.0
New Customer      13.208375  1.0  99999.0
Regular Customer  12.583071  1.0  99999.0

### The mean price of products purchased by loyal customers is lower than that of new and regular customers.

## 06.  Create a spending flag for each user based on the average price across all their orders using the following criteria: 
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [16]:
# Create column mean_price
ords_prods_old['mean_price'] = ords_prods_old.groupby(['user_id'])['prices'].transform(np.mean)

In [17]:
ords_prods_old.head()

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,Busiest Days,busiest_period_of_day,max_order,loyalty_flag,mean_price
0,2539329,1,1,2,8,11.0,196,1,0,both,Soda,77,7,9.0,Mid-Range Product,Regularly Busy,Average Orders,10,Regular Customer,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797


In [18]:
# Create a spending_flag 
ords_prods_old.loc[ords_prods_old['mean_price'] < 10, 'spending_flag'] = 'Low Spender'

In [19]:
ords_prods_old.loc[ords_prods_old['mean_price'] >= 10, 'spending_flag'] = 'High Spender'

In [20]:
# Check output 
ords_prods_old['spending_flag'].value_counts(dropna = False)

Low Spender     31770614
High Spender      634245
Name: spending_flag, dtype: int64

In [21]:
# Check output 
ords_prods_old.head(100)

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,price_range_loc,Busiest Days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag
0,2539329,1,1,2,8,11.0,196,1,0,both,...,77,7,9.0,Mid-Range Product,Regularly Busy,Average Orders,10,Regular Customer,6.367797,Low Spender
1,2398795,1,2,3,7,15.0,196,1,1,both,...,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender
2,473747,1,3,3,12,21.0,196,1,1,both,...,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797,Low Spender
3,2254736,1,4,4,7,29.0,196,1,1,both,...,77,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender
4,431534,1,5,4,15,28.0,196,1,1,both,...,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797,Low Spender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3226575,360,1,5,12,11.0,196,1,0,both,...,77,7,9.0,Mid-Range Product,Regularly Busy,Most Orders,3,New Customer,10.006250,High Spender
96,1469869,377,3,5,17,3.0,196,9,0,both,...,77,7,9.0,Mid-Range Product,Regularly Busy,Average Orders,3,New Customer,8.496552,Low Spender
97,1927023,387,2,4,10,22.0,196,3,0,both,...,77,7,9.0,Mid-Range Product,Least Busy,Most Orders,8,New Customer,7.396610,Low Spender
98,858092,420,4,1,19,30.0,196,2,0,both,...,77,7,9.0,Mid-Range Product,Busiest Days,Average Orders,22,Regular Customer,7.387805,Low Spender


## 07. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows: 
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [22]:
# Create column days_median showing the median number of days since each user's prior order 
ords_prods_old['days_median'] = ords_prods_old.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [23]:
# Check output
ords_prods_old.head(50)

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,department_id,prices,price_range_loc,Busiest Days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,days_median
0,2539329,1,1,2,8,11.0,196,1,0,both,...,7,9.0,Mid-Range Product,Regularly Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0
1,2398795,1,2,3,7,15.0,196,1,1,both,...,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0
2,473747,1,3,3,12,21.0,196,1,1,both,...,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0
3,2254736,1,4,4,7,29.0,196,1,1,both,...,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0
4,431534,1,5,4,15,28.0,196,1,1,both,...,7,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0
5,3367565,1,6,2,7,19.0,196,1,1,both,...,7,9.0,Mid-Range Product,Regularly Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0
6,550135,1,7,1,9,20.0,196,1,1,both,...,7,9.0,Mid-Range Product,Busiest Days,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0
7,3108588,1,8,1,14,14.0,196,2,1,both,...,7,9.0,Mid-Range Product,Busiest Days,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0
8,2295261,1,9,1,16,0.0,196,4,1,both,...,7,9.0,Mid-Range Product,Busiest Days,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0
9,2550362,1,10,4,8,30.0,196,1,1,both,...,7,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0


In [24]:
# Create order_frequency flags
ords_prods_old.loc[ords_prods_old['days_median'] > 20, 'order_frequency_flag'] = 'Non-frequent Customer'

In [25]:
ords_prods_old.loc[ords_prods_old['days_median'] < 10, 'order_frequency_flag'] = 'Frequent Customer'

In [26]:
ords_prods_old.loc[(ords_prods_old['days_median'] <= 20) & (ords_prods_old['days_median'] >= 10), 'order_frequency_flag'] = 'Regular Customer'

In [27]:
# Check value counts
ords_prods_old['order_frequency_flag'].value_counts(dropna = False)

Frequent Customer        19294222
Regular Customer         10410377
Non-frequent Customer     2700260
Name: order_frequency_flag, dtype: int64

In [28]:
# Check output 
ords_prods_old.head(50)

,order_id,user_id,order_number,order_day_of_week,order_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,prices,price_range_loc,Busiest Days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,days_median,order_frequency_flag
0,2539329,1,1,2,8,11.0,196,1,0,both,...,9.0,Mid-Range Product,Regularly Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
1,2398795,1,2,3,7,15.0,196,1,1,both,...,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
2,473747,1,3,3,12,21.0,196,1,1,both,...,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
3,2254736,1,4,4,7,29.0,196,1,1,both,...,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
4,431534,1,5,4,15,28.0,196,1,1,both,...,9.0,Mid-Range Product,Least Busy,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
5,3367565,1,6,2,7,19.0,196,1,1,both,...,9.0,Mid-Range Product,Regularly Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
6,550135,1,7,1,9,20.0,196,1,1,both,...,9.0,Mid-Range Product,Busiest Days,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
7,3108588,1,8,1,14,14.0,196,2,1,both,...,9.0,Mid-Range Product,Busiest Days,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
8,2295261,1,9,1,16,0.0,196,4,1,both,...,9.0,Mid-Range Product,Busiest Days,Most Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer
9,2550362,1,10,4,8,30.0,196,1,1,both,...,9.0,Mid-Range Product,Least Busy,Average Orders,10,Regular Customer,6.367797,Low Spender,20.0,Regular Customer


In [29]:
# Check results of only three columns 
ords_prods_old[['user_id','days_median','order_frequency_flag']].head(50)

,user_id,days_median,order_frequency_flag
0,1,20.0,Regular Customer
1,1,20.0,Regular Customer
2,1,20.0,Regular Customer
3,1,20.0,Regular Customer
4,1,20.0,Regular Customer
5,1,20.0,Regular Customer
6,1,20.0,Regular Customer
7,1,20.0,Regular Customer
8,1,20.0,Regular Customer
9,1,20.0,Regular Customer


## 09. Export to Pickle format


In [30]:
ords_prods_old.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4.8_task_flags.pkl'))